## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-08-16-51-49 +0000,cbc,Anti-hate bill stalled after Fraser’s office b...,https://www.cbc.ca/news/politics/liberals-anti...
1,2025-12-08-16-51-11 +0000,wapo,Paramount launches hostile bid for Warner Bros...,https://www.washingtonpost.com/business/2025/1...
2,2025-12-08-16-47-12 +0000,nyt,Storm Bram’s Heavy Rain and Strong Winds Could...,https://www.nytimes.com/2025/12/08/weather/sto...
3,2025-12-08-16-45-09 +0000,nyt,How Japan Built a Rare-Earth Supply Chain With...,https://www.nytimes.com/2025/12/08/business/ja...
4,2025-12-08-16-44-15 +0000,nyt,One Hundred Schoolchildren Released After Kidn...,https://www.nytimes.com/2025/12/08/world/afric...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
102,trump,36
16,warner,16
238,new,15
7,deal,14
19,netflix,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
48,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...,124
206,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...,97
86,2025-12-08-14-06-59 +0000,nyt,Trump Looms Over Netflix’s Deal for Warner Bro...,https://www.nytimes.com/2025/12/08/business/de...,95
141,2025-12-08-10-44-44 +0000,bbc,Trump says $72bn Netflix-Warner Bros deal 'cou...,https://www.bbc.com/news/articles/cn815egjqjpo...,94
99,2025-12-08-13-20-35 +0000,nypost,‘Disappointed’ Trump claims Zelensky ‘hasn’t y...,https://nypost.com/2025/12/08/world-news/disap...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
48,124,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...
99,51,2025-12-08-13-20-35 +0000,nypost,‘Disappointed’ Trump claims Zelensky ‘hasn’t y...,https://nypost.com/2025/12/08/world-news/disap...
76,39,2025-12-08-14-51-32 +0000,nypost,New winter storm stretching as far south as No...,https://nypost.com/2025/12/08/us-news/new-wint...
150,38,2025-12-08-10-03-11 +0000,nyt,Supreme Court to Hear Major Test of Trump’s Po...,https://www.nytimes.com/2025/12/08/us/politics...
235,34,2025-12-07-21-44-45 +0000,cbc,Conservative motion will force Liberals to 'pu...,https://www.cbc.ca/news/politics/poilievre-con...
208,33,2025-12-08-02-19-24 +0000,bbc,Former Australian deputy PM joins anti-immigra...,https://www.bbc.com/news/articles/cqxq8g4d873o...
253,32,2025-12-07-20-18-00 +0000,nypost,SL Green chief expects thriving NYC office ma...,https://nypost.com/2025/12/07/business/sl-gree...
194,32,2025-12-08-04-15-18 +0000,nyt,China’s Trade Surplus Climbs Past $1 Trillion ...,https://www.nytimes.com/2025/12/07/business/ch...
71,31,2025-12-08-14-56-20 +0000,nypost,7.2-magnitude earthquake strikes off Japan’s c...,https://nypost.com/2025/12/08/world-news/7-2-m...
96,28,2025-12-08-13-28-00 +0000,wsj,The Trump administration is planning to announ...,https://www.wsj.com/politics/policy/trump-to-u...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
